# Figures for DSHARP Opacity paper

## General import for all figures

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib

#from disklab import opacity
import dsharp_opac as opacity

import aux_functions as aux
aux.set_style()

## Figure 1: comparison of optical constants

In [ ]:
# get the optical constants

water_henning = opacity.diel_henning('waterice',      new=True)
water_pollack = opacity.diel_henning('waterice',      new=False)
water_warren = opacity.diel_warrenbrandt08()

organics_henning = opacity.diel_henning('organics',      new=True)
organics_pollack = opacity.diel_henning('organics',      new=False)

troilite_henning = opacity.diel_henning('troilite',      new=True)
troilite_pollack = opacity.diel_henning('troilite',      new=False)

labels = ['Henning & Stognienko (1996)', 'Pollack et al. (1994)', 'Warren & Brandt (2008)']

const_w = [
    water_henning,
    water_pollack,
    water_warren
    ]

const_o = [
    organics_henning,
    organics_pollack
    ]

const_t = [
    troilite_henning,
    troilite_pollack
    ]

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(3.5, 3 * 3.5 * 0.54), sharex=True)

axs = axs.ravel()
axs2 = []
for dc, ax in zip([const_w, const_o, const_t], axs):

    lines_n = []
    lines_k = []
    
    # plot real part
    for _c, _l in zip(dc, labels):
        lines_n += [ax.semilogx(_c._l, _c._n, '-', alpha=0.7, label=_l)[0]]

    # plot imaginary part

    ax2 = ax.twinx()
    axs2 +=[ax2]
    for _c, _l in zip(dc, labels):
        lines_k += [ax2.loglog(_c._l, _c._k, '--', alpha=0.7, label=_l)[0]]

    # add name of the species

    ax.text(.05,.9,
            dc[0].material_str.split()[0],
            horizontalalignment='left',
            transform=ax.transAxes)

    # add legend to last panel

    if len(axs2)==1:
        lines_n += [ax.plot([], [], c='k', ls='-',  label='$n$')[0]]
        lines_n += [ax.plot([], [], c='k', ls='--', label='$k$')[0]]
        ax.legend(lines_n, [l.get_label() for l in lines_n], loc=4, fontsize='x-small')

# adjust the labels

axs[-1].set_xlabel('wavelength [cm]')

for ax in axs[:-2]:
    plt.setp(ax.get_xticklabels(), visible=False)

for ax in axs:
    ax.set_ylabel('n')
    
for ax in axs2:
    ax.set_ylabel('k')

# use same limits as in pollack paper

axs[0].set_ylim(0.8,2.0)
axs2[0].set_ylim(1e-9,1e1)

axs[1].set_ylim(1.4,2.4)
axs2[1].set_ylim(1e-3,1e0)

axs[2].set_ylim(1e-2,1e3)
axs[2].set_yscale('log')
axs2[2].set_ylim(2e-2,1e3)


axs[0].set_xlim(1e-5, 1e1)

f.subplots_adjust(
    hspace=0.1, wspace=0.0,
    bottom=0.08, top=0.94,
    left=0.15, right=0.85)

f.savefig('../figures/fig1_optical_constants.pdf')

## Figure 2: mixed optical constants

In [ ]:
diel, rho_s = opacity.get_dsharp_mix(rule='Bruggeman')
print('Average material density = {:.4f} g/cc'.format(rho_s))

diel = diel.get_normal_object()

f, ax = plt.subplots(figsize=(3.5, 3.5 * 0.615))


lines = [ax.semilogx(diel._l, diel._n, '-', alpha=0.7, label='n')[0]]
ax2 = ax.twinx()
lines += [ax2.loglog(diel._l, diel._k, '--', alpha=0.7, label='k')[0]]

# add legend to last panel
ax.legend(lines, [l.get_label() for l in lines], loc=7, fontsize='small')

ax.set_xlabel('wavelength [cm]')
ax.set_ylabel('n')
ax2.set_ylabel('k')
ax.set_xlim(1e-5, 1e1)

f.subplots_adjust(
    bottom=0.18, top=0.96,
    left=0.11, right=0.88
    )

f.savefig('../figures/fig2_mix.pdf')

## Figure 3: opacities

Loading the data

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(3.5, 3.5 * 2 * 0.51), sharex=True, sharey=True)

ax = axs[0]
cc = ax.pcolormesh(a, lam, np.log10(k_abs), vmin=-5, vmax=5, rasterized=True)
cb = plt.colorbar(cc, ax=ax, extend='both', shrink=0.95, pad=0.02)
cb.set_label(r'$\log_{10} \, \kappa_\nu^\mathrm{abs} \, [\mathrm{cm}^2/\mathrm{g}$]')

ax = axs[1]
cc = ax.pcolormesh(a, lam, np.log10(k_sca), vmin=-5, vmax=5, rasterized=True)
cb = plt.colorbar(cc, ax=ax, extend='both', shrink=0.95, pad=0.02)
cb.set_label(r'$\log_{10} \, \kappa^\mathrm{sca}_\nu \, [\mathrm{cm}^2/\mathrm{g}$]')

for ax in axs:
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylabel('$\lambda$ [cm]')

axs[0].get_yticklabels()[1].set_visible(False)
axs[1].set_xlabel('$a$ [cm]')

f.subplots_adjust(
    left=0.15,  right=0.99,
    bottom=0.11, top=0.96,
    hspace=0)
f.savefig('../figures/fig3_opacity.pdf', dpi=300)

## Figure 4: Default Size Averaging

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
lam_avg = [0.1, 0.3]
q       = [3.5, 2.5]
res     = [opacity.size_average_opacity(lam_avg, a, lam, k_abs.T, k_sca.T, q=_q, plot=False) for _q in q]

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(3.5, 3.5 * 2 * 0.60), sharex=True)
i_lam = 0

for _res,_q in zip(res,q):
    l, = axs[0].loglog(a, _res['ka'][i_lam], '-', label='$q$ = {}, absorption'.format(_q))
    axs[0].loglog(a, _res['ks'][i_lam], '--', label='$q$ = {}, scattering'.format(_q), c=l.get_color())
    
    axs[1].plot(a, _res['beta'], '-', label='$q$ = {}'.format(_q), c=l.get_color())
    
axs[0].set_ylabel('$\kappa_\mathrm{{{:.0f}\,mm}}$ [cm$^2$/g]'.format(10 * lam_avg[i_lam]))
axs[1].set_ylabel(r'$\beta_\mathrm{{{:.0f}-{:.0f}\,mm}}$'.format(10 * lam_avg[0], 10 * lam_avg[1]))
axs[1].set_xlabel(r'$a_\mathrm{max}$ [cm]')
    
axs[0].legend(fontsize='x-small', loc=2)
axs[1].legend(fontsize='x-small', loc=2)
axs[0].set_ylim(1e-2, 1e2)
axs[1].set_ylim(0, 3.6)
axs[1].set_xlim(1e-4, 1e2)

f.subplots_adjust(
    left=0.13, right=0.97,
    bottom=0.09, top=0.98,
    wspace=0.1, hspace=0.05)

f.savefig('../figures/fig4_size_average.pdf')

## Figure 5 & 6: Size Averaged $\lambda$ dependence, Size Distributions

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
# Download one of the Weingartner & Draine 2001 dust models for comparison

filename = 'kext_albedo_WD_MW_3.1_60'
if not os.path.isfile(os.path.join('data', filename)):
    from urllib.request import urlretrieve
    print('Downloading file \'{}\' ... '.format(filename), end='')
    urlretrieve(
        'ftp://ftp.astro.princeton.edu/draine/dust/mix/{}'.format(filename),
        filename=os.path.join('data', filename))
    print('Done!')

with open(os.path.join('data', filename)) as f:
    while not f.readline().startswith('--------'):
        pass
    lam_dr,_,_,_,k_dr = np.loadtxt(f).T

In [ ]:
# a 1g-normalized size distribution (bin-integrated) up to 1 mm

s = a**0.5
s[a > 0.1] = 0
s= s / s.sum()

# size average the absorption opacities

k_powerlaw  = (k_abs.T * s[:,None]).sum(0)

Create a coagulation-fragmentation size distribution with similar upper size limit.

In [ ]:
r = c.au.cgs.value          # position in the disk in cm
T = 200.                    # temperature in K
sigma_g = 200.              # gas surface density [g/cm^2]
d2g = 0.01                  # dust-to-gas ratio
rho_s = 1.6686              # material density [g/cm^3]
M_star = c.M_sun.cgs.value  # stellar mass [g]
v_frag = 100.               # fragmentation velocity [cm/s]
alpha = 4e-4                # turbulence parameter

m = 4 * np.pi / 3 * rho_s * a**3  # mass grid

# create the size distribution, FIRST: the fortran code

fit, a_01, a_12, a_l, a_p, a_r, a_sett = opacity.distribution(1.8, T, alpha, sigma_g, sigma_g * d2g, rho_s, m, a, M_star, r, v_frag)

# SECOND: a simplified version. Need to convert to bin-integrated values.

fit2, a_f = aux.get_new_fit(T, a, r=r, sigma_g=sigma_g, d2g=d2g, rho_s=rho_s, M_star=M_star, v_frag=v_frag, alpha=alpha)
fit2 = fit2 * np.diff(np.hstack((a[0]**2 / a[1], a)))
fit2 = fit2 / fit2.sum() * sigma_g * d2g

# plot the size distribution

fig, ax = plt.subplots(figsize=(3.5, 3.5 * 0.66))
ax.loglog(a, s * sigma_g * d2g, label='power-law distribution')
ax.loglog(a, fit, label='B11')
ax.loglog(a, fit2, label='B11 $-$ simplified')
ax.set_xlim(1e-5, 2e-1)
ax.set_ylim(1e-4, 2e-1)
ax.set_xlabel('particle size [cm]')
ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
ax.legend(fontsize='small')

fig.subplots_adjust(
    bottom=0.17, top=0.95,
    left=0.14, right=0.97
    )
fig.savefig('../figures/fig5_size_distri.pdf')

In [ ]:
# size average the absorption opacities
s2 = fit / fit.sum()
k_fit  = (k_abs.T * s2[:,None]).sum(0)

s3 = aux.get_new_fit(T, a)[0]
s3 *= a
s3 = s3 / s3.sum()
k_fit2  = (k_abs.T * s3[:,None]).sum(0)

In [ ]:
# where to measure the reference value
lam_obs = 0.087

# load the D'Alessio opacity
d2g = sum([0.0056, 0.0034, 0.000768, 0.0041])
data_d01 = np.loadtxt('data/kappa_D01_T100K_p3.5_amax1mm.csv')
lam_d01 = 10.**data_d01[:,0] * 1e-4
kap_d01 = 10.**(data_d01[:,1]) / d2g

In [ ]:
# the Beckwith 1990 law

kb = 3.5 * (lam / 0.087)**(-1)  # Beckwith 1990

# the opacities from Andrews et al. 2009

la, ka = np.loadtxt('data/andrews2009.dat').T

# now the plot

f, ax = plt.subplots(figsize=(3.5, 3.5 * 0.66))

ax.loglog(1e-4*lam_dr, k_dr,       'k',   zorder=-100, alpha=0.5, label='Weingartner & Draine 2001')
ax.loglog(lam,         kb,         'k--', zorder=-100, alpha=0.5, label='Beckwith et al. 1990')
ax.loglog(1e-4*la,     ka,         'k--', zorder=-100, alpha=1.0, label='Andrews et al. 2009')
ax.loglog(lam_d01,     kap_d01,    'k:',  zorder=-100, alpha=1.0, label='D\'Alessio et al. 2001')
l1, = ax.loglog(lam,   k_powerlaw,        zorder=-100, alpha=1.0, label='DSHARP, power-law')
l2, = ax.loglog(lam,   k_fit,             zorder=-100, alpha=1.0, label='DSHARP, B11 fit', ls='--', color=l1.get_color())
l2, = ax.loglog(lam,   k_fit2,             zorder=-100, alpha=1.0, label='DSHARP, B11 simple fit', ls=':', color=l1.get_color())

ax.legend(loc=3, fontsize='xx-small').get_frame().set_edgecolor('none')
ax.set_xlim(1e-5,1e0)
ax.set_ylim(1e-2,1e5)
ax.set_xlabel('$\lambda$ [cm]')
ax.set_ylabel(r'$\kappa^\mathrm{abs,tot}_\nu$ [cm$^2$/g]')

f.subplots_adjust(
    bottom=0.171, top=0.94,
    left=0.15, right=0.97)

plt.savefig('../figures/fig6_comparison.pdf')

## Fig. 7: comparison of the size distributions

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
gsca   = d['g'].T
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
r      = c.au.cgs.value                # position in the disk in cm
TEMP   = [100, 200., 500]              # temperature in K
SIG_G  = [100, 200., 500]              # gas surface density [g/cm^2]
d2g    = 0.01                          # dust-to-gas ratio
rho_s  = 1.6686                        # material density [g/cm^3]
M_star = c.M_sun.cgs.value             # stellar mass [g]
V_FRAG = [100., 1000, 2000]            # fragmentation velocity [cm/s]
ALPHA  = [1e-4, 1e-3, 1e-2]            # turbulence parameter
m      = 4 * np.pi / 3 * rho_s * a**3  # mass grid

params = aux.permutate_fiducial([TEMP, SIG_G, V_FRAG, ALPHA])
labels = [r'T = {:g} K', '$\Sigma_\mathrm{{g}} = {:g}$ g/cm$^2$', '$v_\mathrm{{frag}}$ = {:g} m/s', r'$\alpha = 10^{{{:g}}}$']


f, axs = plt.subplots(4, 1, figsize=(3.5, 4 * 3.5 * 0.575), sharex=True, sharey=True)
axs = axs.flat


for i_param in range(len(params[0])):
    ax = axs[i_param]
    
    params_slice = [params[-1], params[2 * i_param], params[2 * i_param + 1]]
    
    params_slice = sorted(params_slice, key = lambda o: o[i_param])
    
    for values in params_slice:
        
        T, sig_g, v_frag, alpha = values

        # create the size distribution, FIRST: the fortran code

        fit, a_01, a_12, a_l, a_p, a_r, a_sett = opacity.distribution(1.8, T, alpha, sig_g, sig_g * d2g, rho_s, m, a, M_star, r, v_frag)

        # SECOND: a simplified version. Need to convert to bin-integrated values.

        fit2, a_f = aux.get_new_fit(T, a, r=r, sigma_g=sig_g, d2g=d2g, rho_s=rho_s, M_star=M_star, v_frag=v_frag, alpha=alpha)
        fit2 = fit2 * np.diff(np.hstack((a[0]**2 / a[1], a)))
        fit2 = fit2 / fit2.sum() * sig_g * d2g
        
        # plotting
        
        val = values[i_param]
        if i_param==2:
            val /=100
        if i_param==3:
            val = np.log10(val)
        
        l1, = ax.loglog(a, fit, label=labels[i_param].format(val))
        l2, = ax.loglog(a, fit2, '--', c=l1.get_color())
        
    ax.loglog([], [], 'k--', label='simple fit')
    ax.legend(fontsize='x-small', loc=2)
    ax.set_ylabel('$\sigma$ [g cm$^{-2}$]')
        
ax.set_xlim(1e-5, 2e2)
ax.set_ylim(6e-5, 2e-1)
ax.set_xlabel('particle size [cm]')

f.subplots_adjust(
    bottom=0.05, top=0.98,
    left=0.14, right=0.97,
    hspace=0.05
    )

f.savefig('../figures/fig7_distri_panels.pdf')

# Fig. 8: Mean Opacities with fit distributions

In [ ]:
d      = np.load('data/default_opacities_smooth.npz')
a      = d['a']
gsca   = d['g'].T
lam    = d['lam']
k_abs  = d['k_abs']
k_sca  = d['k_sca']

In [ ]:
# define temperature and wavelength arrays

T_array = np.logspace(0, np.log10(1500), 250)
nu = c.c.cgs.value / lam

# calculate the planck function and their derivatives for all those temperatures and wavelength

Bnu    = np.array([aux.planck_B_nu(nu, T) for T in T_array])
dBnudT = np.array([aux.planck_dBnu_dT(nu, T) for T in T_array])

## Calculate mean opacities for all temperatures

For the Rosseland Mean, we need the *extinction opacity*

$$
\kappa_\nu^\mathrm{ext}(a) = \kappa_\nu^{abs}(a) + (1-g)\,\kappa_\nu^{sca}(a)
$$

then the total opacity of the size distribution is given by

$$
\kappa_\nu^\mathrm{abs,tot} = \frac{\int_0^{a_\mathrm{max}} n(a)\,m\,\kappa_\nu^\mathrm{abs}(a) \,\mathrm{d}a}{\int_0^{a_\mathrm{max}} n(a)\,m \,\mathrm{d}a}
$$

The Planck mean opacity is 

$$
\bar \kappa_\mathrm{P}(T) = \frac{\int_0^\infty \kappa_\nu^\mathrm{abs,tot}\, B_\nu(T)\,\mathrm{d}\nu}{\int_0^\infty  B_\nu(T)\,\mathrm{d}\nu}
$$

and the Rosseland mean opacity is

$$
\bar \kappa_\mathrm{R}(T) = \left( \frac{\int_0^\infty \frac{1}{\kappa_\nu^\mathrm{ext,tot}} \, \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}{\int_0^\infty  \frac{\mathrm{d}B_\nu(T)}{\mathrm{d}T}\,\mathrm{d}\nu}\right)^{-1}
$$

In [ ]:
k_ext = k_abs + (1 - gsca) * k_sca

In [ ]:
q = 3.5
amax = 0.1
power_law = a**(4 - q)
power_law[a > amax] = 0
power_law = power_law / power_law.sum()

fits = np.zeros([len(T_array), len(a)])

k_P_p = np.zeros_like(T_array)
k_P_f = np.zeros_like(T_array)
k_R_p = np.zeros_like(T_array)
k_R_f = np.zeros_like(T_array)

# for each temperature ...

for it, T in enumerate(T_array):
    
    # get the size distribution fit
    
    f = aux.get_fit(T, a)
    f = f / f.sum()    
    fits[it, :] = f
    
    # sum the EXTINCTION opacity
    
    k_abs_p = (k_abs * power_law[None, :]).sum(1)
    k_abs_f = (k_abs * f[None, :]).sum(1)
    
    # sum the EXTINCTION opacity
    
    k_ext_p = (k_ext * power_law[None, :]).sum(1)
    k_ext_f = (k_ext * f[None, :]).sum(1)
    
    # calculate Planck opacity for the fit and the power-law
    
    B   = np.trapz(Bnu[it, :], x=nu)
    k_P_p[it] = np.trapz(Bnu[it, :] * k_abs_p, x=nu) / B
    k_P_f[it] = np.trapz(Bnu[it, :] * k_abs_f, x=nu) / B
    
    # calculate Rosseland opacity for the fit and the power-law
    
    B   = np.trapz(dBnudT[it, :], x=nu)
    k_R_p[it] = B / np.trapz(dBnudT[it, :] / k_ext_p, x=nu)
    k_R_f[it] = B / np.trapz(dBnudT[it, :] / k_ext_f, x=nu)

#### The spline fit to one of the curves

In [ ]:
from scipy.interpolate import LSQUnivariateSpline, BSpline, splev

k = 2 # order of the spline
s1 = LSQUnivariateSpline(np.log10(T_array), np.log10(k_P_f), [0.5, 1, 2, 2.5, 3], k=k)

knots = s1.get_knots()
knots = k*[knots[0]] + list(knots) + k*[knots[-1]]
coeff = s1.get_coeffs()

print('knots        = '+ ', '.join(['{:6.3g}'.format(_k) for _k in knots]))
print('coefficients = '+ ', '.join(['{:6.3g}'.format(_k) for _k in coeff]))

s2 = BSpline(knots, coeff, k)

#### Do the plot

In [ ]:
f, ax = plt.subplots(figsize=(3.5, 3.5 * 0.65))
l1, = ax.loglog(T_array, k_P_p, label='Planck mean, power-law')
l2, = ax.loglog(T_array, k_R_p, label='Rosseland mean, power-law')
l3, = ax.loglog(T_array, k_P_f, '--', c=l1.get_color(), label='Planck mean, B11')
l4, = ax.loglog(T_array, k_R_f, '--', c=l2.get_color(), label='Rosseland mean, B11')

#ax.loglog(T_array, 10.**(s2(np.log10(T_array))), 'k--', alpha=0.5)

ax.legend(fontsize='small')
ax.set_xlabel('$T$ [K]')
ax.set_ylabel(r'$\bar\kappa$ [cm$^2$/g]')

f.subplots_adjust(
    left=0.14, right=0.98,
    bottom=0.17, top=0.97
    )
f.savefig('../figures/fig8_mean_opacities.pdf')